In [119]:
import pandas as pd
import numpy as np

In [120]:
bb_train = pd.read_csv('../data/Annotation/boundingbox/bb_train.csv', index_col='id')
bb_test = pd.read_csv('../data/Annotation/boundingbox/bb_test.csv', index_col='id')
manual_train = pd.read_csv('../data/Annotation/manual/filled_manual_train.csv', index_col='id')
manual_test = pd.read_csv('../data/Annotation/manual/filled_manual_test.csv', index_col='id')
# manual_train = pd.read_csv('../data/Annotation/auto/auto_train.csv', index_col='id')
# manual_test = pd.read_csv('../data/Annotation/auto/auto_test.csv', index_col='id')

In [121]:
train_join = manual_train.join(bb_train)
test_join = manual_test.join(bb_test)

In [122]:
train_join

,manual_1_X,manual_1_Y,manual_2_X,manual_2_Y,manual_3_X,manual_3_Y,manual_4_X,manual_4_Y,manual_5_X,manual_5_Y,Gender,Race,Age,bb_1_pt1,bb_1_pt2,bb_1_pt3,bb_1_pt4
id,,,,,,,,,,,,,,,,,
3396,49.717,52.668,66.047,53.490,56.588,63.745,46.746,65.425,66.696,68.856,0,0,2,38.242939,31.359859,74.649460,78.644226
4548,263.424,256.442,345.468,230.839,357.105,317.538,277.971,379.216,342.558,368.743,1,0,2,105.271286,107.973785,365.024323,438.301300
3383,153.938,168.343,222.093,173.324,189.314,209.578,152.332,237.299,215.898,239.318,1,0,3,111.392014,92.749931,250.398315,287.923401
11406,48.938,49.313,68.188,53.125,56.813,65.000,46.188,69.250,63.250,71.813,0,0,2,37.502903,31.234165,76.062981,81.005188
11413,347.057,377.877,480.759,377.908,391.157,441.117,347.146,526.761,467.286,524.382,0,0,3,277.475494,210.256927,563.360474,612.510742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,38.238,46.854,53.133,42.181,45.102,54.886,48.314,60.435,57.076,57.660,1,0,2,33.351357,23.345911,71.284134,67.461609
11365,47.230,53.162,66.246,52.639,59.442,63.106,48.931,72.090,63.186,71.966,1,0,1,35.804913,33.686436,73.521980,80.800629
11370,180.272,205.277,261.871,200.166,220.109,252.183,197.122,290.871,253.196,286.607,1,0,2,137.374619,120.728912,303.003326,336.257202


In [123]:
X_cols = [f'manual_{i}_X' for i in range(1, 6)]
Y_cols = [f'manual_{i}_Y' for i in range(1, 6)]

categorical_cols = ['Gender', 'Race', 'Age']

def standardize(df):
    X_df = df[X_cols].subtract(df['bb_1_pt1'], axis=0).divide(df['bb_1_pt3'] - df['bb_1_pt1'], axis=0)
    Y_df = df[Y_cols].subtract(df['bb_1_pt2'], axis=0).divide(df['bb_1_pt4'] - df['bb_1_pt2'], axis=0)
    categorical_df = df[categorical_cols]
    return X_df.join(Y_df).join(categorical_df)

In [104]:
# X_cols = [f'auto_{i}_X' for i in range(1, 37)]
# Y_cols = [f'auto_{i}_Y' for i in range(1, 37)]

# def standardize(df):
#     X_df = df[X_cols].subtract(df['bb_1_pt1'], axis=0).divide(df['bb_1_pt3'] - df['bb_1_pt1'], axis=0)
#     Y_df = df[Y_cols].subtract(df['bb_1_pt2'], axis=0).divide(df['bb_1_pt4'] - df['bb_1_pt2'], axis=0)
#     return X_df.join(Y_df)

In [124]:
std_train = standardize(train_join)
std_test = standardize(test_join)

In [125]:
#read true labels
labels = pd.read_table('../data/basic/EmoLabel/list_patition_label.txt', sep=' ', header=None)

In [126]:
#separate train and test labels
train_labels = {}
test_labels = {}
for index, row in labels.iterrows():
    if 'train' in row[0]:
        split_id = row[0].split('_')[1].split('.')[0]
        train_labels.update({int(split_id): row[1]})
    elif 'test' in row[0]:
        split_id = row[0].split('_')[1].split('.')[0]
        test_labels.update({int(split_id): row[1]})
        
train_classes = [-1]*len(train_labels)
for k, v in train_labels.items():
    train_classes[int(k)-1] = v

test_classes = [-1]*len(test_labels)
for k, v in test_labels.items():
    test_classes[int(k)-1] = v

In [127]:
len(train_labels)

12271

In [128]:
len(test_labels)

3068

In [129]:
std_train['label'] = pd.Series(train_labels)
std_test['label'] = pd.Series(test_labels)

In [130]:
std_train_dummies = pd.get_dummies(std_train, columns=['Gender', 'Race', 'Age'])
std_test_dummies = pd.get_dummies(std_test, columns=['Gender', 'Race', 'Age'])

In [131]:
from sklearn.linear_model import LogisticRegression

In [132]:
clf = LogisticRegression(max_iter=500).fit(std_train_dummies.drop(columns=['label']),
                               std_train_dummies['label'])

In [133]:
clf.score(std_test_dummies.drop(columns=['label']), std_test_dummies['label'])

0.5371577574967406

In [115]:
# clf = LogisticRegression(max_iter=1000).fit(std_train.drop(columns=['label']),
#                                std_train['label'])

In [116]:
# clf.score(std_test.drop(columns=['label']), std_test['label'])

0.5945241199478487

In [134]:
# std_train.to_csv('../data/Annotation/auto/auto_std_train.csv')
# std_test.to_csv('../data/Annotation/auto/auto_std_test.csv')

In [ ]:
std_train.to_csv('../data/Annotation/manual/manual_std_train.csv')
std_test.to_csv('../data/Annotation/manual/manual_std_test.csv')